In [ ]:
import apache_beam as beam

p = beam.Pipeline()

black = ('Adão','Jesus','Mike')
White = ('Tulio','Mary','Joca')
first_nations = ('Vic','Marta','Tom')

black_pc = p | "Creating Pcollection black" >> beam.Create(black)
White_pc = p | "Creating Pcollection White" >> beam.Create(White)
first_nations_pc = p | "Creating Pcollection first_nations" >> beam.Create(first_nations)

people = (
    (black_pc,White_pc,first_nations_pc) 
        | beam.Flatten()
        | beam.Map(print))
p.run()

In [19]:
import apache_beam as beam
import os

# Define the input and output file paths
input_file = '../datasets/transactions.csv'
output_file_prefix = '../datasets/output/transactions_output_headers'

# Function to calculate total amount spent and format the result
def calculate_total_amount(record):
    # Split the input record into fields
    fields = record.split(',')
    customer_id = fields[0]
    product_id = fields[1]
    quantity = int(fields[2])
    price = float(fields[3])
    
    # Calculate total amount spent
    total_amount = quantity * price
    
    # Return the record with total amount included
    return f'{customer_id},{product_id},{quantity},{price},{total_amount}'

# Create a pipeline
with beam.Pipeline() as p:
    # Read data from the input file
    data = (
        p
        | 'ReadFromText' >> beam.io.ReadFromText(input_file)
        # Apply the Map transformation to calculate total amount
        | 'CalculateTotalAmount' >> beam.Map(calculate_total_amount)
    )

    # Define the header
    header = p | 'CreateHeader' >> beam.Create(['CustomerID,ProductID,Quantity,Price,TotalAmount'])

    # Combine the header with the data
    result = (
        (header, data)
        | 'CombineHeaderAndData' >> beam.Flatten()
        # Write the combined result to the output file
        | 'WriteToText' >> beam.io.WriteToText(output_file_prefix, file_name_suffix='.csv')
    )

# No need for p.run() as it's implicitly handled by the context manager
